<h1>Smart Tour Guide Bot</h1>

**Project Title:**
Multimodal AI Travel Assistant

**Names:**

- Renad Naser
- ⁠Amjad Althagafi

**Objective:**

In a world with many transportations and countries, many people are becoming excited about the visiting new places and discover new things. 
Nowadays, the internet is full with many resources which make it difficult to find a specific information with being sure that it is correct or enough.

So we design and deploy an intelligent AI chatbot that answers user travel-related questions using voice or text, by extracting and summarizing data from global travel guide sources.

**Description:**

This project aims to build a multimodal RAG chatbot that allows users to ask questions and requests, like —“What can I visit in Paris in 2 days?”—and receive instant, accurate responses. The chatbot uses speech recognition, and summarize relevant information from tourist guides like Wikivoyage. It supports both text and voice input and provides responses using pre-trained transformer models.

**AI Techniques:**

	•	Speech Recognition (Whisper by OpenAI)
	•	Text Chunking & Embedding (LangChain + HuggingFace Transformers)
	•	Vector Search (ChromaDB or FAISS DB)
	•	Multimodal RAG system
	•	Conversational Agents with Memory (LangChain Agents)
	•	LangSmith for evaluation

**Applications:**

	•	Smart travel planning assistants
	•	Accessibility tools for visually/hearing-impaired travelers
	•	Educational tools for learning about geography and culture
	•	Chatbots for tourism websites or mobile travel apps

In [2]:
pip install speechrecognition datasets faiss-cpu PyAudio

Note: you may need to restart the kernel to use updated packages.


In [3]:
# travel_assistant.py
import os
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import load_dataset
import speech_recognition as sr

In [4]:

# Initialize environment
load_dotenv()
recognizer = sr.Recognizer()

In [5]:
# 1. Data Processing and Vector Store Setup
def initialize_knowledge_base():
    embed = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    if not os.path.exists("faiss.index"):
        # Load and process data
        data = load_dataset("wikipedia", "20220301.simple", split="train[:100]", trust_remote_code=True)
        
        def create_document(article):
            return Document(
                page_content=article["text"],
                metadata={"title": article["title"]}
            )
        
        documents = [create_document(raw) for raw in data]
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        split_docs = splitter.split_documents(documents)
        
        # Create and save vector store
        vectorstore = FAISS.from_documents(split_docs, embed)
        vectorstore.save_local("faiss.index")
    
    # Load existing vector store
    return FAISS.load_local("faiss.index", embed, allow_dangerous_deserialization=True)


In [6]:
def voice_to_text():
    try:
        with sr.Microphone() as source:
            print("\nListening... (Speak now)")
            audio = recognizer.listen(source, timeout=10)
        return recognizer.recognize_google(audio)
    except sr.WaitTimeoutError:
        print("Listening timed out")
        return ""
    except Exception as e:
        print(f"Audio error: {str(e)}")
        return ""

def get_user_input():
    input_type = input("\nChoose input method [text/audio]: ").lower()
    return voice_to_text() if input_type == "audio" else input("Your question: ")


In [7]:
# 3. Response Formatting
def format_response(response):
    formatted = f"\nANSWER: {response['result']}"
    sources = {doc.metadata['title'] for doc in response['source_documents']}
    if sources:
        formatted += "\n\nSources:\n- " + "\n- ".join(sources)
    return formatted

In [8]:
# 4. Main Application
def main():
    # Initialize components
    vectorstore = initialize_knowledge_base()
    qa_chain = RetrievalQA.from_chain_type(
        llm=OpenAI(temperature=0.2, model_name="gpt-3.5-turbo-instruct"),
        retriever=vectorstore.as_retriever(),
        return_source_documents=True
    )

    print("\n" + "="*50)
    print("🌟 Travel Assistant Activated 🌟")
    print("Ask about any city or request travel plans!")
    print("Examples:")
    print("- What's special about Tokyo?")
    print("- 3-day itinerary for Rome")
    print("- Tell me about New York's museums")
    print("="*50)

    while True:
        try:
            # Get user input
            query = get_user_input()
            if not query:
                continue

            # Process query
            if any(kw in query.lower() for kw in ["plan", "itinerary", "schedule"]):
                response = qa_chain.invoke({"query": f"Create detailed travel plan: {query}"})
            else:
                response = qa_chain.invoke({"query": query})

            # Show response
            print(format_response(response))

        except KeyboardInterrupt:
            print("\n🚗 Safe travels! Closing assistant...")
            break

if __name__ == "__main__":
    main()


🌟 Travel Assistant Activated 🌟
Ask about any city or request travel plans!
Examples:
- What's special about Tokyo?
- 3-day itinerary for Rome
- Tell me about New York's museums

Listening... (Speak now)

ANSWER:  I don't know.

Sources:
- Afghanistan
- Algebra
- Armenia
- Belgium

Listening... (Speak now)

ANSWER:  London is a metropolis that includes several small ancient towns and villages, including London, Westminster, and many old villages such as Notting Hill, Southwark, Richmond, Greenwich, etc. The part that is officially known as the "City of London" only takes up one square mile, while the rest is known as "Greater London." It is one of the world's largest cities and has been continually changing for hundreds of years.

Sources:
- April
- City

Listening... (Speak now)
Audio error: 

🚗 Safe travels! Closing assistant...


In [ ]:
pip install PyAudio

Note: you may need to restart the kernel to use updated packages.
